In [1]:
from fablr import dataframes as d

In [2]:
from fablr.extended_providers import ToManyProvider, ToOneProvider

In [3]:
gen = d.Fablr()
gen.set_seed(123)

In [4]:
from datetime import datetime as dt
date_format = '%Y-%m-%d'
column_providers_dict = {
  'first_name': {'provider': 'first_name', 'kwargs': {}},
  'last_name': {'provider': 'last_name', 'kwargs': {}},
  'email': {'provider': 'email', 'kwargs': {}},
  'last_login': {'provider': 'date_between_dates',
                 'kwargs': {'date_start': dt.strptime('2023-01-01', date_format), 'date_end': dt.strptime('2023-12-01', date_format)}},
  #'foo': {'provider': 'foo', 'kwargs': {}},
}

In [5]:
data = gen.generate_dataframe(10, column_providers_dict, ["first_name", "last_name"])
data.head(5)

,first_name,last_name,email,last_login,hash
0,Brian,Miller,cadams@example.com,2023-07-28,ff9874715f4ceca27f1603e1babc6e80779d5949e35ba1...
1,Anna,Peterson,moorematthew@example.com,2023-09-30,9f781215b0ebae5f53f97a13bd93a4a53652a33f9a56d9...
2,John,Smith,mayala@example.com,2023-01-28,898a445987bfd157aff2af28a78a91c30679a8590ce267...
3,Adam,Dennis,whiteheadchristopher@example.org,2023-02-19,d668225fc0938701c806631ae28b5097cc2a27ab90924a...
4,Kristy,Smith,qberry@example.com,2023-11-22,c631fad82a156f5cfdcdee525837ea68fa3bfd82afc6e0...


In [6]:
gen.fake.add_provider(ToManyProvider)
gen.fake.add_provider(ToOneProvider)

In [7]:
[gen.fake.ToManyValue(data, "first_name") for x in range(0,11)]

['Jose',
 'Benjamin',
 'Brian',
 'Adam',
 'Jose',
 'Anna',
 'Benjamin',
 'Brian',
 'Benjamin',
 'Adam',
 'Annette']

In [8]:
[gen.fake.ToOneValue(data, "first_name") for x in range(0,10)]

['Anthony',
 'Brian',
 'Kristy',
 'Gail',
 'Adam',
 'John',
 'Annette',
 'Benjamin',
 'Anna',
 'Jose']

In [14]:
gen.clear()

AttributeError: 'Generator' object has no attribute 'ToOneProvider'

In [10]:
data["first_name"].to_list()

['Brian',
 'Anna',
 'John',
 'Adam',
 'Kristy',
 'Benjamin',
 'Annette',
 'Gail',
 'Anthony',
 'Jose']

In [11]:
gen.fake.unique.random_int(min=0, max = 5)

0

In [18]:
gen.fake.ToOneValue.clear()

AttributeError: 'function' object has no attribute 'clear'

In [13]:
gen.fake.unique.clear()